In [1]:
from sklearn import tree
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [319]:
#функция преобразования шкалы к стенам - 1-10
def sten(x):
    return (5.5 + 2* (x - x.mean())/x.std()).apply(lambda x: 10 if x>10 else x).astype(int) 

In [606]:
VUZ_dict = {'p1_aw20_amursu_student':'Амурский ГУ', 
    'p1_aw20_asau_student':'Алтайский ГАУ',
    'p1_aw20_bsau_student':'Башкирский ГАУ',
    'p1_aw20_volgsau_student':'Волгоградский ГАУ',
    'p1_aw20_vsau_student':'Воронежский ГАУ',
    'p1_aw20_vstu_student':'Воронежский ГТУ',
    'p1_aw20_vstu_student':'Воронежский ГТУ',
    'p1_aw20_tsaa_student':'ГАУ Северного Зауралья',
    'p1_aw20_dvfu_student':'Дальневосточный федеральный университет (ДВФУ)',
    'p1_aw20_donstu_student':'Донской ГТУ',
    'p1_aw20_ivgsha_student':'Ивановская ГСХА',
    'p1_aw20_ksu_edu_student':'Костромской ГУ',
    'p1_aw20_kubsau_student':'Кубанский ГАУ',
    'p1_aw20_ksai_student':'Кузбасская ГСХА',
    'p1_aw20_kgsha_student':'Курская ГСХА',
    'p1_aw20_mitumasi_student':'МИТУ МАСИ',
    'p1_aw20_mgau_student':'Мичуринский ГАУ',
    'p1_aw20_midis_student':'Международный Институт Дизайна  Сервиса',
    'p1_aw20_itmo_student':'Национальный исследовательский университет ИТМО',
    'p1_aw20_ngma_student':'НИМИ им. Кортунова',
    'p1_aw20_nsau_edu_student':'Новосибирский ГАУ',
    'p1_aw20_omgau_student':'Омский ГАУ',
    'p1_aw20_osu_student':'Оренбургский ГУ',
    'p1_aw20_orelsau_student':'Орловский ГАУ',
    'p1_aw20_penzgau_student':'Пензенский ГАУ',
    'p1_aw20_pgsha_student':'Пермский ГАТУ',
    'p1_aw20_psuti_student':'Поволжский ГУТИ',
    'p1_aw20_pgusa_student':'Приамурский ГУ им.Шолом-Алейхема',
    'p1_aw20_timacad_student':'РГАУ-МСХА им. Тимирязева',
    'p1_aw20_rsue_student':'РГЭУ РИНХ',
    'p1_aw20_rgatu_student':'Рязанский ГАТУ имени П.А. Костычева',
    'p1_aw20_samgau_student':'Самарский ГАУ',
    'p1_aw20_samsmu_student':'Самарский ГМУ',
    'p1_aw20_ssau_student':'Самарский НИУ им. академика С.П. Королева',
    'p1_aw20_sgau_student':'Саратовский ГАУ имени Н.И. Вавилова',
    'p1_aw20_sevsu_student':'СевГУ',
    'p1_aw20_ncfu_student':'Северо-Кавказский федеральный университет',
    'p1_aw20_spbgau_student':'СПбГАУ',
    'p1_aw20_leti_student':'СПбГЭТУ "ЛЭТИ"',
    'p1_aw20_stsau_student':'Ставропольский ГАУ',
    'p1_aw20_tstu_student':'Тамбовский ГТУ',
    'p1_aw20_ulsau_student':'Ульяновский ГАУ',
    'p1_aw20_chuvsau_student':'Чувашский ГАУ',
    'p1_aw20_chuvsu_student':'Чувашский ГУ им. Ульянова',
    'p1_aw20_ugrasu_student':'Югорский государственный универитет',
    'p1_aw20_sfedu_student':'ЮФУ',
    'p1_aw20_sursau_student':'Южно-Уральский ГАУ'}

In [600]:
engagement_source=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Engagement/P1_aw_engagement_2_2.csv', \
                              sep=';',encoding='utf-8')
course_source=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Engagement/P1_aw_courses_2_2.csv',sep=';',encoding='utf-8')

In [601]:
course_summ = course_source[course_source.progress>0].groupby(['untiID','period_id']).agg(
    course_progress_count=('course_id', 'count'),
    course_progress_sum=('progress', 'sum')
) 

engagement_merged = pd.merge(
    engagement_source,
    course_summ,
    left_on=['untiID','period_id'],
    right_index=True,
    how='left' ).fillna(0)

course_point = engagement_merged.sort_values(['untiID','period_id'])
# NA  обозначаем как -1
course_point_diff = course_point.groupby('untiID') \
    [['course_enrolled','course_finished','course_progress_count','course_progress']]. \
    diff().reset_index().fillna(-1)

#Сшиваем теперь показатели прогресса (показатели периода- показатели периода-1) и
engagement_merged = pd.merge(
    engagement_merged,
    course_point_diff,
    left_index=True,
    right_index=True,
    how='left' ).fillna(0)

engagement_merged = engagement_merged.rename(columns={'course_enrolled_y':'enroll_delta',
                                                     'course_finished_y':'finish_delta',
                                                     'course_progress_count_y':'p_c_delta' ,
                                                     'course_progress_y':'progress_delta'})
engagement_merged.head()

,untiID,leaderID,VUZ_tag,period_id,cards_created,cards_moved,course_enrolled_x,course_finished_x,course_progress_x,event_request,...,favour_count,collections,project_likes,course_progress_count_x,course_progress_sum,index,enroll_delta,finish_delta,p_c_delta,progress_delta
0,106128,633496,p1_aw20_rsue_student,1,0,0,1,0,0,0,...,0,0,0,0.0,0.0,8,-1.0,-1.0,-1.0,-1.0
1,106128,633496,p1_aw20_rsue_student,2,0,0,1,0,0,0,...,0,0,0,0.0,0.0,9,0.0,0.0,0.0,0.0
2,106794,706995,p1_aw20_ncfu_student,1,0,0,1,0,0,0,...,0,0,0,0.0,0.0,10,-1.0,-1.0,-1.0,-1.0
3,106794,706995,p1_aw20_ncfu_student,2,0,0,1,0,0,0,...,0,0,0,0.0,0.0,11,0.0,0.0,0.0,0.0
4,109005,759363,p1_aw20_rsue_student,1,0,4,1,1,0,0,...,0,0,0,0.0,0.0,4656,-1.0,-1.0,-1.0,-1.0


In [602]:
report_students = pd.DataFrame({'untiID': [], 'leaderID':[], 'VUZ':[], 
                                'Trello': [], 'Courses': [],'Events': [], 'Step': [], 'Social': [], 
                                'Total': [], 'live':[], 'period':[] })
report_students.untiID   = engagement_merged.untiID
report_students.leaderID = engagement_merged.leaderID
report_students.VUZ      = engagement_merged.VUZ_tag
report_students.Trello   = engagement_merged.cards_created*3 + engagement_merged.cards_moved*6
report_students.Courses  = engagement_merged.apply(lambda x: x['course_enrolled_x']*2 if x['enroll_delta']==-1   \
                                                       else x['enroll_delta']*2, axis=1) + \
                           engagement_merged.apply(lambda x: x['course_finished_x']*10 if x['finish_delta']==-1  \
                                                       else x['finish_delta']*10, axis=1) + \
                           engagement_merged.apply(lambda x: x['course_progress_count_x']*4 if x['p_c_delta']==-1 \
                                                       else x['p_c_delta']*4, axis=1) + \
                           engagement_merged.apply(lambda x: 1 if x['progress_delta']>0 and x['period_id']>1 \
                                                       else  0, axis=1) 
report_students.Events   = engagement_merged.event_request *2 + engagement_merged.event_checkin*2 + \
                           engagement_merged.feedback_points + engagement_merged.artefacts*5
report_students.Step     = engagement_merged.focus_count * 3 + engagement_merged.step_full *3 + \
                           engagement_merged.like_count + engagement_merged.collections*10
report_students.Social   = engagement_merged.project_likes
report_students.Total    = report_students.Trello + report_students.Courses + report_students.Events + \
                           report_students.Step + report_students.Social
report_students.live     = report_students.Total.apply(lambda x: 1 if x>0 else 0) 
report_students.period   = engagement_merged.period_id
report_students.head(10)

,untiID,leaderID,VUZ,Trello,Courses,Events,Step,Social,Total,live,period
0,106128,633496,p1_aw20_rsue_student,0,2.0,0,0,0,2.0,1,1
1,106128,633496,p1_aw20_rsue_student,0,0.0,0,0,0,0.0,0,2
2,106794,706995,p1_aw20_ncfu_student,0,2.0,0,0,0,2.0,1,1
3,106794,706995,p1_aw20_ncfu_student,0,0.0,0,0,0,0.0,0,2
4,109005,759363,p1_aw20_rsue_student,24,12.0,0,0,0,36.0,1,1
5,109005,759363,p1_aw20_rsue_student,0,0.0,0,0,0,0.0,0,2
6,114659,1028283,p1_aw20_timacad_student,0,24.0,0,0,0,24.0,1,1
7,114659,1028283,p1_aw20_timacad_student,0,0.0,7,0,0,7.0,1,2
8,12450,313745,p1_aw20_spbgau_student,0,14.0,6,0,0,20.0,1,1
9,12450,313745,p1_aw20_spbgau_student,0,0.0,0,0,0,0.0,0,2


In [603]:
VUZ_Total_live = report_students[report_students.live==1].groupby(['VUZ','period'])['Total']. \
          describe()[['50%','75%']]

report_students_quartile = pd.merge(
    report_students,
    VUZ_Total_live,
    right_index = True,
    #on='VUZ',
    left_on=['VUZ','period'],
    how='left').fillna(0)

#Переименуем ВУЗы
report_students_quartile.VUZ=report_students_quartile.VUZ.replace(VUZ_dict)
#Переименуем колонки
report_students_quartile= report_students_quartile.rename(columns={'VUZ': 'ВУЗ',
                                          'period': 'Период',
                                          'Trello': 'Работа в Trello',
                                          'Courses':'Online курсы',
                                          'Events':'Мероприятия',
                                          'Step':'Шаги развития',
                                          'Social':'Социальная активность',
                                          'Total':'Итого',
                                          'live':'Живой',
                                          '50%':'Медиана ВУЗа',
                                          '75%':'Третий квартиль ВУЗа',                         
                                         })
#Запишем в файл
report_students_quartile.to_csv('C:/Users/d.muravlyansky/Documents/Python/report/P1_aw_engagement_students_2_2.csv', \
                                encoding='ANSI')  

In [619]:
VUZ_stat = report_students_quartile.groupby(['ВУЗ','Период']).aggregate(
    trello=('Работа в Trello','sum'),
    course=('Online курсы','sum'),
    events=('Мероприятия','sum'),
    step=  ('Шаги развития','sum'),
    social=('Социальная активность', 'sum'),
    Total= ('Итого','sum'),
    total_student=('untiID', 'count'),
    live_student=('Живой','sum')
).fillna(0)

VUZ_active = report_students_quartile[report_students_quartile['Итого']>report_students_quartile['Медиана ВУЗа']/2]. \
    groupby(['ВУЗ','Период']). \
    agg(total_active=('untiID', 'count')).fillna(0)
VUZ_stat= VUZ_stat.join(VUZ_active)

VUZ_Total=report_students_quartile[report_students_quartile['Живой']==1].groupby(['ВУЗ','Период'])['Итого']. \
          describe()[['50%','mean']].fillna(0)
VUZ_stat= VUZ_stat.join(VUZ_Total)

VUZ_stat['trello_per_live'] = sten(VUZ_stat.trello/VUZ_stat.live_student.apply(lambda x: 1 if x==0 else x))
VUZ_stat['course_per_live'] = sten(VUZ_stat.course/VUZ_stat.live_student.apply(lambda x: 1 if x==0 else x))
VUZ_stat['events_per_live'] = sten(VUZ_stat.events/VUZ_stat.live_student.apply(lambda x: 1 if x==0 else x))
VUZ_stat['step_per_live']   = sten(VUZ_stat.step/VUZ_stat.live_student.apply(lambda x: 1 if x==0 else x))
VUZ_stat['social_per_live'] = sten(VUZ_stat.social/VUZ_stat.live_student.apply(lambda x: 1 if x==0 else x))
VUZ_stat['Total_per_live']  = sten(VUZ_stat.Total/VUZ_stat.live_student.apply(lambda x: 1 if x==0 else x))

VUZ_stat=VUZ_stat.fillna(0)
VUZ_stat.head(22)

trello  course  events  \
ВУЗ                                            Период                           
Алтайский ГАУ                                  1            0   128.0     149   
                                               2            0    40.0       8   
Амурский ГУ                                    1            0     0.0      37   
                                               2            0    48.0      16   
Башкирский ГАУ                                 1            0    12.0       2   
                                               2            0    26.0       0   
Волгоградский ГАУ                              1            0   118.0     258   
                                               2            0     2.0      78   
Воронежский ГАУ                                1            0   254.0     207   
                                               2            0    16.0      45   
Воронежский ГТУ                                1         1089   256.0     470   
                                               2          168   314.0     187   
ГАУ Северного Зауралья                         1            0   342.0      16   
                                               2            0     4.0       4   
Дальневосточный федеральный университет (ДВФУ) 1           63    42.0     120   
                                               2          186   224.0      97   
Донской ГТУ                                    1            0   228.0      14   
                                               2            0    38.0       0   
Ивановская ГСХА                                1            0    34.0       4   
                                               2            0    36.0       0   
Костромской ГУ                                 1            0     0.0       0   
                                               2            0    16.0       0   

                                                       step  social   Total  \
ВУЗ                                            Период                         
Алтайский ГАУ                                  1          0       0   277.0   
                                               2          0       0    48.0   
Амурский ГУ                                    1          0       0    37.0   
                                               2          0       0    64.0   
Башкирский ГАУ                                 1          2       0    16.0   
                                               2          0       0    26.0   
Волгоградский ГАУ                              1          3       0   379.0   
                                               2          0       0    80.0   
Воронежский ГАУ                                1          0       0   461.0   
                                               2          0       0    61.0   
Воронежский ГТУ                                1          0       0  1815.0   
                                               2          0       0   669.0   
ГАУ Северного Зауралья                         1          0       0   358.0   
                                               2          0       0     8.0   
Дальневосточный федеральный университет (ДВФУ) 1          0       0   225.0   
                                               2          0       0   507.0   
Донской ГТУ                                    1          0       0   242.0   
                                               2          0       0    38.0   
Ивановская ГСХА                                1          0       0    38.0   
                                               2          0       0    36.0   
Костромской ГУ                                 1          0       0     0.0   
                                               2          0       0    16.0   

                                                       total_student  \
ВУЗ                                            Период                  
Алтайский ГАУ                                  1                  24 

trello  course  events  \
ВУЗ                                            Период                           
Алтайский ГАУ                                  1            0   128.0     149   
                                               2            0    40.0       8   
Амурский ГУ                                    1            0     0.0      37   
                                               2            0    48.0      16   
Башкирский ГАУ                                 1            0    12.0       2   
                                               2            0    26.0       0   
Волгоградский ГАУ                              1            0   118.0     258   
                                               2            0     2.0      78   
Воронежский ГАУ                                1            0   254.0     207   
                                               2            0    16.0      45   
Воронежский ГТУ                                1         1089   256.0     470   
                                               2          168   314.0     187   
ГАУ Северного Зауралья                         1            0   342.0      16   
                                               2            0     4.0       4   
Дальневосточный федеральный университет (ДВФУ) 1           63    42.0     120   
                                               2          186   224.0      97   
Донской ГТУ                                    1            0   228.0      14   
                                               2            0    38.0       0   
Ивановская ГСХА                                1            0    34.0       4   
                                               2            0    36.0       0   
Костромской ГУ                                 1            0     0.0       0   
                                               2            0    16.0       0   

                                                       step  social   Total  \
ВУЗ                                            Период                         
Алтайский ГАУ                                  1          0       0   277.0   
                                               2          0       0    48.0   
Амурский ГУ                                    1          0       0    37.0   
                                               2          0       0    64.0   
Башкирский ГАУ                                 1          2       0    16.0   
                                               2          0       0    26.0   
Волгоградский ГАУ                              1          3       0   379.0   
                                               2          0       0    80.0   
Воронежский ГАУ                                1          0       0   461.0   
                                               2          0       0    61.0   
Воронежский ГТУ                                1          0       0  1815.0   
                                               2          0       0   669.0   
ГАУ Северного Зауралья                         1          0       0   358.0   
                                               2          0       0     8.0   
Дальневосточный федеральный университет (ДВФУ) 1          0       0   225.0   
                                               2          0       0   507.0   
Донской ГТУ                                    1          0       0   242.0   
                                               2          0       0    38.0   
Ивановская ГСХА                                1          0       0    38.0   
                                               2          0       0    36.0   
Костромской ГУ                                 1          0       0     0.0   
                                               2          0       0    16.0   

                                                       total_student  \
ВУЗ                                            Период                  
Алтайский ГАУ                                  1                  24 

In [620]:
VUZ_stat.rename(columns={ 'trello': 'Работа в Trello',
                         'course':'Online курсы',
                         'events':'Мероприятия',
                         'step':'Шаги развития',
                         'social':'Социальная активность',
                         'Total':'Итого',
                         'total_student': 'Всего студентов',
                         'live_student': 'Живых студентов',
                         'total_active': 'Активных студентов',
                         '50%':'Медианная вовлеченность',
                         'mean':'Средняя вовлеченность',
                         'trello_per_live': 'Мощность работы с проектом в Trello',
                         'course_per_live' : 'Мощность работы с Online курсами',
                         'events_per_live' : 'Мощность работы с образовательными мероприятиями',
                         'step_per_live' : 'Мощность работы с шагами развития в Steps',
                         'social_per_live' : 'Мощность социальной активности студентов',
                         'Total_per_live' :'Относительная мощность вуза на интенсиве за период'
                        },
               index=VUZ_dict). \
to_csv('C:/Users/d.muravlyansky/Documents/Python/report/P1_aw_engagement_VUZ_stat_2_2.csv',encoding='ANSI')  